In [1]:
import os
import sys
import math
import tqdm
import wandb
import torch
import logging

import numpy as np
import pandas as pd
import torch.nn as nn
import lightning.pytorch as pl

from torchmetrics import MeanAbsoluteError
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from lightning.pytorch.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import Callback, LearningRateMonitor, ModelCheckpoint

/var/folders/xz/1q44kbwx25g7j6b7qsjs_x4r0000gp/T/ipykernel_31658/2726006082.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig,
    FTTransformerConfig,
    TabNetModelConfig,
    GatedAdditiveTreeEnsembleConfig,
    TabTransformerConfig,
    AutoIntConfig
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

# TabTransformer

https://arxiv.org/abs/2012.06678
https://github.com/manujosephv/pytorch_tabular/blob/main/examples/PyTorch%20Tabular%20with%20Bank%20Marketing%20Dataset.ipynb

In [13]:
project_name = "MSU_interpol"

logger_path = './wandb_local_logs'
data_path = '../data/clasdb_pi_plus_n.txt'

hyperparams_dict = {
    'scale_data': False,
    'augment': True,
    'augment_factor': 20,
    'test_size': 0.1,
    'batch_size': 256,
    'net_architecture': [5,60,80,100,120,140,240,340,440,640,2000,1040,640,340,240,140,100,80,60,20,1],
    'activation_function': nn.ReLU(),
    'optim_func': torch.optim.Adam,
    'max_epochs': 2000,
    'es_patience': 20,
    'auto_lr_find': True
}

In [14]:
df = pd.read_csv(data_path, delimiter='\t', header=None)
df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']
df.loc[8314:65671, 'Ebeam'] = 5.754 # peculiarity of this dataset.
df['phi'] = df.phi.apply(lambda x: math.radians(x))
df = df.drop(['id', 'error'], axis=1)
df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]

train, test = train_test_split(df,
                              test_size=hyperparams_dict.get('test_size'),
                              random_state=1438)



In [15]:
data_config = DataConfig(
    target=['dsigma_dOmega'],
    continuous_cols=['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']
)

trainer_config = TrainerConfig(
    auto_lr_find=hyperparams_dict.get('auto_lr_find'),
    batch_size=hyperparams_dict.get('batch_size'),
    max_epochs=hyperparams_dict.get('max_epochs'),
    early_stopping="valid_loss",
    early_stopping_mode = "min",
    early_stopping_patience=hyperparams_dict.get('es_patience'),
    checkpoints="valid_loss",
    load_best=True
)

optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

In [17]:
model_config = TabTransformerConfig(
    task="regression",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)
tabular_model.fit(train=train)
tabular_model.evaluate(test)

2024-02-05 20:51:08,476 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2024-02-05 20:51:08,507 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-02-05 20:51:08,523 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

2024-02-05 20:51:08,573 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: TabTransformerModel

...

2024-02-05 20:53:58,413 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

Output()

/Users/andrey.golda/Library/Caches/pypoetry/virtualenvs/msu-interpol--lw2ADYE-py3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.8117692470550537     │
│  test_mean_squared_error  │    1.8117692470550537     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.8117692470550537,
  'test_mean_squared_error': 1.8117692470550537}]